# Predicting Heart Failure

#### *Pedro Arrizon, Justin Callahan, Jennice Herrera, Doeun(Leah) Lee, Abhinav Mugunda*

##### [Github](https://github.com/P-Arrizon/SDS322E_Project)

# The Goal

The goal for this project is to try and use statistical methods and data science methods to try and dicover some realtionships within our data that can help us realize any signs of heart complications given some data. 

# The data: